<a href="https://colab.research.google.com/github/mateusduarte-max/Projetos_Python/blob/main/Fraude_Pre%C3%A7o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Bibliotecas Google Golab
from google.colab import auth
auth.authenticate_user()
# Importar biblioteca biqquery
from google.cloud import bigquery
# Bibliotecas Google Sheet
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
from gspread_dataframe import set_with_dataframe
# Montar Drive
from google.colab import drive
drive.mount('/content/drive')
# Classe dados 
!cp /content/drive/MyDrive/Colab_Notebooks/pacotes_modulos/modulos_colab.py /content
from modulos_colab import dados
!cp /content/drive/MyDrive/Colab_Notebooks/Autenticacoes/chaves_tokens.py /content
%run chaves_tokens.py

# Análise de Preço por Pedido

In [ ]:
df_pedidos = dados.consulta_bigquery('maga-bigdata',
                                     """
                                     WITH
                                        pedidos AS(
                                        SELECT
                                            PARSE_DATE('%Y%m%d', CAST(datapedidokey AS STRING)) as data_pedido,
                                            VCAP.IDSELLERMARKETPLACE AS IDSELLERMARKETPLACE,
                                            VCAP.produtomarketplacekey AS produtomarketplacekey,
                                            categoria,
                                            subcategoria,
                                            IFNULL(VCAP.quantidadeitens,0) AS qtd_itens,
                                            numeropedidocliente,
                                            round(SAFE_DIVIDE(sum(VCAP.VLVENDAITEM), sum(VCAP.quantidadeitens)),2) as VLVENDAITEM,
                                            round(IFNULL(VCAP.vlfreteclientenominal,0),2) AS VLFRETE,
                                            
                                        FROM
                                            maga-bigdata.apolo.vw_fact_vendas_captadas VCAP
                                        WHERE
                                            PARSE_DATE('%Y%m%d', CAST(datapedidokey AS STRING)) >= CURRENT_DATE()-1
                                            --AND DATE(VCAP.datapagamentomktp) IS NULL
                                            AND UPPER(VCAP.STATUS) <> 'TESTE'
                                            AND UPPER (IFNULL(VCAP.IDSELLERMARKETPLACE,'-1')) <> 'MAGAZINELUIZA'
                                        GROUP BY
                                            1,
                                            2,
                                            3,
                                            4,
                                            5,
                                            6,
                                            7,
                                            9),

                                        media_cat_subcat as (
                                            SELECT
                                            categoria as cat_media,
                                            subcategoria as subcat_media,
                                            --produtomarketplacekey,
                                            --numeropedidocliente, 
                                            round(AVG(vlvendaitem),2) as valor_medio_cat_subcat

                                        FROM
                                            `maga-bigdata.apolo.vw_fact_vendas_captadas`
                                        where 
                                            PARSE_DATE('%Y%m%d', CAST(datapedidokey AS STRING)) >= current_date()-15
                                            --and numeropedidocliente in ('1235570425274502', '1235570425273204', '1235570425269271')
                                            --and categoria in ('AR','CO')
                                            --and subcategoria in ('ARSP','CMBX')
                                        group by
                                            1,2

                                        order by
                                            1 
                                            asc
                                        ),

                                        safra as (
                                            select distinct 
                                            SellerID,
                                            Data_Publicado,
                                            Carteira,
                                            Status,
                                            Plataforma
                                        from 
                                            `maga-bigdata.bi_ecomm.TB_SafraSellers`
                                        ),
                                        
                                        catalogo AS (
                                            SELECT distinct
                                        navigation_id,
                                        title,
                                        date(CREATED_AT) as data_sku
                                        FROM
                                        maga-bigdata.catalogo.visao_catalogo_produto_atual 
                                            )

                                        

                                        SELECT DISTINCT 
                                            *  EXCEPT(cat_media , subcat_media,SellerID,navigation_id ),
                                            safe_divide(VLVENDAITEM, valor_medio_cat_subcat)*100 as IP_SKU
                                        FROM 
                                            PEDIDOS AS P
                                        LEFT JOIN 
                                            media_cat_subcat AS C
                                        ON 
                                        P.categoria=C.cat_media 
                                        AND 
                                            P.subcategoria=C.subcat_media
                                        LEFT JOIN SAFRA AS S 
                                        ON
                                            LOWER(S.SellerID)=LOWER(P.IDSELLERMARKETPLACE)
                                        LEFT JOIN catalogo as ca
                                        ON ca.navigation_id=P.produtomarketplacekey
                                        WHERE
                                            --IDSELLERMARKETPLACE='telmababymoveis'
                                            safe_divide(VLVENDAITEM, valor_medio_cat_subcat) <= 0.60
                                            and s.Carteira='Autosserviço'
                                            and VLVENDAITEM > 200
                                            --and categoria in ('ED','MO','ET','TE','ES','RC','AR','AU','PI','CO','PF','FJ','EP','CJ','BB','IN','EA','CP','BR')
                                            
                                
                                        ORDER BY
                                            Data_Publicado
                                            DESC

                                     """)


df_pedidos[['VLVENDAITEM','VLFRETE','valor_medio_cat_subcat','IP_SKU']] = round(df_pedidos[['VLVENDAITEM','VLFRETE','valor_medio_cat_subcat','IP_SKU']].astype(float),2)
df_pedidos[['VLVENDAITEM','VLFRETE','valor_medio_cat_subcat','IP_SKU']] = df_pedidos[['VLVENDAITEM','VLFRETE','valor_medio_cat_subcat','IP_SKU']].astype(str)

update = (datetime.now() - timedelta(hours=3)).strftime("%d/%m/%Y %H:%M:%S")


df_pedidos['Update_planilha'] = update

dados.grava_tabela_drive("ANÁLISE_FRAUDE_PEDIDO","Análise_Preço_Pedido",df_pedidos)
print('Tabela Atualizada!')

# Análise de Preço no Cadastro (Alteração Preço)

In [ ]:
# Formatar data (retroagir 2 dias)
data_inicial = (datetime.now() -timedelta(days=2)).strftime("%Y%m%d")

# Consulta BQ
df_pedidos = dados.consulta_bigquery('maga-bigdata', f"""
                                                WITH
                                                    cat_hoje AS (
                                                    SELECT
                                                        DISTINCT 
                                                        DATETIME(c.created_at) AS created_at,
                                                        c.seller_id,
                                                        c.sku,
                                                        c.navigation_id,
                                                        c.niagara_timestamp AS niagara_timestamp,
                                                        c.brand,
                                                        cat.id AS categoria,
                                                        sub.id AS subcategoria,
                                                        c.offer_title AS titulo,
                                                        ROW_NUMBER() OVER (PARTITION BY c.seller_id, c.sku, c.navigation_id, c.brand, cat.id, sub.id, c.offer_title ORDER BY c.niagara_timestamp DESC) AS RN,
                                                        p.price
                                                    FROM
                                                        `maga-bigdata.catalogo.product_*` AS c,
                                                        UNNEST (c.categories) cat,
                                                        UNNEST(cat.subcategories) sub
                                                    LEFT JOIN
                                                        `maga-bigdata.pricing.updatedprice_*` AS p
                                                    ON
                                                        p.seller_id=c.seller_id
                                                        AND p.sku=c.sku
                                                    WHERE
                                                        --DATE(created_at)=CURRENT_DATE()
                                                        c._table_suffix >= '{data_inicial}'
                                                        and p._table_suffix >= '{data_inicial}'


                                                    GROUP BY
                                                        1,
                                                        2,
                                                        3,
                                                        4,
                                                        5,
                                                        6,
                                                        7,
                                                        8,
                                                        9,
                                                        11),

                                                    media_cat_subcat AS (
                                                    SELECT
                                                        categoria AS cat_media,
                                                        subcategoria AS subcat_media,
                                                        --produtomarketplacekey,
                                                        --numeropedidocliente,
                                                        ROUND(AVG(vlvendaitem),2) AS valor_medio_cat_subcat
                                                    FROM
                                                        `maga-bigdata.apolo.vw_fact_vendas_captadas`
                                                    WHERE
                                                        PARSE_DATE('%Y%m%d', CAST(datapedidokey AS STRING)) >= CURRENT_DATE()-15 
                                                        
                                                    GROUP BY
                                                    1,
                                                    2
                                                    ORDER BY
                                                    1 ASC ),


                                                    safra AS (
                                                    SELECT
                                                        DISTINCT SellerID,
                                                        Data_Publicado,
                                                        Carteira,
                                                        Status,
                                                        Plataforma
                                                    FROM
                                                        `maga-bigdata.bi_ecomm.TB_SafraSellers` )


                                                    SELECT
                                                    * EXCEPT(cat_media,
                                                        subcat_media,
                                                        SellerID),
                                                    SAFE_DIVIDE(price,valor_medio_cat_subcat) AS IP
                                                    FROM
                                                    cat_hoje AS c
                                                    LEFT JOIN
                                                    media_cat_subcat AS m
                                                    ON
                                                    LOWER(c.categoria)=LOWER(m.cat_media)
                                                    AND c.subcategoria=m.subcat_media
                                                    LEFT JOIN
                                                    safra AS s
                                                    ON
                                                    LOWER(s.SellerID)=LOWER(c.seller_id)
                                                    WHERE
                                                    RN=1
                                                    AND SAFE_DIVIDE(price,valor_medio_cat_subcat) <0.70
                                                    AND Carteira NOT IN ('Verticais','Top Sellers','Key Account','Loja do grupo')
                                                    and price >200.00
                                                    --and categoria in ('ED','MO','ET','TE','ES','RC','AR','AU','PI','CO','PF','FJ','EP','CJ','BB','IN','EA','CP','BR')
                                                    
                                                    ORDER BY
                                                    Data_Publicado desc
                                             """)


df_pedidos[['price','PRECO_POR','IP']] = round(df_pedidos[['price','valor_medio_cat_subcat','IP']].astype(float),2)


df_pedidos['Update_planilha'] = update

dados.grava_tabela_drive("ANÁLISE_FRAUDE_PEDIDO","Análise_Preço_Cadastro",df_pedidos)
print('Tabela Atualizada!')

In [ ]:
# Média preço categoria
'''
df_preco_medio_cat = dados.consulta_bigquery('maga-bigdata',
                                             """
                                             SELECT
                                                        categoria AS cat_media,
                                                        subcategoria AS subcat_media,
                                                        --produtomarketplacekey,
                                                        --numeropedidocliente,
                                                        ROUND(AVG(vlvendaitem),2) AS valor_medio_cat_subcat
                                                    FROM
                                                        `maga-bigdata.apolo.vw_fact_vendas_captadas`
                                                    WHERE
                                                        PARSE_DATE('%Y%m%d', CAST(datapedidokey AS STRING)) >= CURRENT_DATE()-15 
                                                        
                                                    GROUP BY
                                                    1,
                                                    2
                                                    ORDER BY
                                                    1 ASC
                                             """)

dados.grava_tabela_drive('ANÁLISE_FRAUDE_PEDIDO', 'preço_medio_categoria', df_preco_medio_cat)

'''